# Data Load & Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

univ_df = pd.read_excel('/content/gdrive/MyDrive/BOOT/code2_final.xlsx') #엑셀 파일로 저장했을 경우
# univ_df = pd.read_csv(csv 파일 명.csv) #csv 파일로 저장했을 경우

# 선형 회귀 모델 학습 및 결과 해석

선형 회귀 모델은 예측력이 높지는 않지만 해석력이 높은 모델이다. 따라서, 영향 분석을 통해 의미를 도출하고자 하는 해당 프로젝트에 적합하다.

지금까지 프로젝트를 진행하면서 각자 도출한 데이터셋도 다를 것이며, 각자가 갖고 있는 인사이트도 다르기에 자유롭게 지금까지 배운 선형 회귀 모델 지식을 활용해 결과를 해석하면 된다.

아래는 선형 회귀 모델을 학습시킬 때에 참고할 수 있는 코드들이다.

## 평균 중심화

평균 중심화(mean centering)는 각 변수에서 그 변수의 평균을 뺌으로써 이루어진다.

In [ ]:
import pandas as pd
import numpy as np
numeric_columns = list(univ_df.select_dtypes(include=['int64', 'float64']).columns)
numeric_columns_all = list(univ_df.select_dtypes(include=['int64', 'float64']).columns)
#가정: 'data'는 평균 중심화를 할 데이터프레임이다

In [ ]:
numeric_columns

['입학정원_학부_계',
 '정원내_입학자_학부_계',
 '정원내_입학자_학부_남',
 '정원내_입학자_학부_여',
 '전임교원_계',
 '모집인원',
 '최종_경쟁률',
 '표준점수',
 '재학생충원율',
 '취업률',
 '신입생충원율',
 '총교육비',
 '학생_1인당_교육비',
 '전임교원수',
 '재학생수',
 '전임교원확보율',
 '장학금총액',
 '학생_1인당_장학금',
 '모집군_가',
 '모집군_나',
 '모집군_다',
 '설립_국가기관',
 '설립_사립',
 '캠퍼스_본교',
 '캠퍼스_분교',
 '전체 재학생수_대',
 '전체 재학생수_소',
 '전체 재학생수_중',
 '계열_인문',
 '계열_자연',
 '학교명',
 '재학생_성비',
 '최종 경쟁률:학생 1인당 장학금',
 '최종 경쟁률:학교명',
 '최종 경쟁률:취업률',
 '설립_국가기관:장학금 총액',
 '설립_사립:장학금 총액',
 '재학생_성비:정원내_입학자_학부_남',
 '학교명:학생_1인당_장학금']

In [ ]:
elements_to_remove = ['전체 재학생수_대','전체 재학생수_중','전체 재학생수_소', '모집군_가', '모집군_나', '모집군_다', '설립_국가기관', '설립_사립', '캠퍼스_본교', '캠퍼스_분교', '학교명','계열_자연','계열_인문']  # 제거할 항목들의 리스트
for element in elements_to_remove:
  numeric_columns.remove(element)
print(numeric_columns)

['입학정원_학부_계', '정원내_입학자_학부_계', '정원내_입학자_학부_남', '정원내_입학자_학부_여', '전임교원_계', '모집인원', '최종_경쟁률', '표준점수', '재학생충원율', '취업률', '신입생충원율', '총교육비', '학생_1인당_교육비', '전임교원수', '재학생수', '전임교원확보율', '장학금총액', '학생_1인당_장학금', '재학생_성비', '최종 경쟁률:학생 1인당 장학금', '최종 경쟁률:학교명', '최종 경쟁률:취업률', '설립_국가기관:장학금 총액', '설립_사립:장학금 총액', '재학생_성비:정원내_입학자_학부_남', '학교명:학생_1인당_장학금']


In [ ]:
centered_data = univ_df[numeric_columns] - np.mean(univ_df[numeric_columns], axis=0)
univ_df[numeric_columns] = centered_data

## 조절 효과 확인

조절 효과를 확인하는 것은 변수들 간의 관계를 조절변수를 통해 파악하는 것을 의미한다. 여기서는 statsmodels의 OLS 클래스를 사용하여 선형 회귀를 실행한다.

In [ ]:
univ_df.columns

Index(['대학명', '대계열', '중계열', '소계열', '학과명', '입학정원_학부_계', '정원내_입학자_학부_계',
       '정원내_입학자_학부_남', '정원내_입학자_학부_여', '전임교원_계', '모집인원', '최종_경쟁률', '표준점수',
       '재학생충원율', '취업률', '신입생충원율', '총교육비', '학생_1인당_교육비', '전임교원수', '재학생수',
       '전임교원확보율', '장학금총액', '학생_1인당_장학금', '학생모집정지여부', '모집군_가', '모집군_나', '모집군_다',
       '설립_국가기관', '설립_사립', '캠퍼스_본교', '캠퍼스_분교', '전체_재학생수_대', '전체_재학생수_소',
       '전체_재학생수_중', '계열_인문', '계열_자연', '학교명', '재학생_성비'],
      dtype='object')

In [ ]:
import statsmodels.api as sm
# 상호작용항 추가
univ_df['설립_국가기관:장학금 총액'] = univ_df['설립_국가기관'] * univ_df['장학금총액']
univ_df['설립_사립:장학금 총액'] = univ_df['설립_사립'] * univ_df['장학금총액']
univ_df['학교명:학생_1인당_장학금'] = univ_df['학교명'] * univ_df['학생_1인당_장학금']
univ_df['학교명:전임교원수'] = univ_df['학교명'] * univ_df['전임교원수']
univ_df['재학생_성비:정원내_입학자_학부_남'] = univ_df['재학생_성비'] * univ_df['정원내_입학자_학부_남']

In [ ]:
# 전체 재학생수 대중소
# 독립변수
X = univ_df[['전체_재학생수_대','전체_재학생수_중','전체_재학생수_소']]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['표준점수']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                   표준점수   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     111.0
Date:                Tue, 22 Aug 2023   Prob (F-statistic):           2.52e-47
Time:                        15:45:25   Log-Likelihood:                -4463.0
No. Observations:                3221   AIC:                             8932.
Df Residuals:                    3218   BIC:                             8950.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0392      0.022     -1.814      0.0

In [ ]:
#최종 경쟁률, 1인당 장학금
# 독립변수
X = univ_df[['최종_경쟁률','학생_1인당_장학금']]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['표준점수']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                   표준점수   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     238.1
Date:                Wed, 23 Aug 2023   Prob (F-statistic):           3.71e-97
Time:                        08:14:14   Log-Likelihood:                -4348.2
No. Observations:                3221   AIC:                             8702.
Df Residuals:                    3218   BIC:                             8721.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.536e-17      0.016  -3.36e-15      1.0

In [ ]:
#최종 경쟁률, 취업
# 독립변수
X = univ_df[['학교명', '전임교원수', '학교명:전임교원수']]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['총교육비']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                   총교육비   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     5861.
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        08:10:39   Log-Likelihood:                -1564.5
No. Observations:                3221   AIC:                             3137.
Df Residuals:                    3217   BIC:                             3161.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3248      0.026     12.552      0.0

In [ ]:
#최종 경쟁률, 1인당 장학금
# 독립변수
X = univ_df[['설립_사립', '장학금총액', '설립_사립:장학금 총액']]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['표준점수']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                   표준점수   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     277.9
Date:                Wed, 23 Aug 2023   Prob (F-statistic):          2.24e-160
Time:                        07:15:27   Log-Likelihood:                -4199.3
No. Observations:                3221   AIC:                             8407.
Df Residuals:                    3217   BIC:                             8431.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.3163      0.028     11.244   

In [ ]:
#최종 경쟁률, 1인당 장학금
# 독립변수
X = univ_df[['설립_국가기관', '장학금총액', '설립_국가기관:장학금 총액']]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['표준점수']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                   표준점수   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     277.9
Date:                Wed, 23 Aug 2023   Prob (F-statistic):          2.24e-160
Time:                        09:10:59   Log-Likelihood:                -4199.3
No. Observations:                3221   AIC:                             8407.
Df Residuals:                    3217   BIC:                             8431.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -0.1509      0.019     -7.

In [ ]:
univ_major = pd.get_dummies(univ_df, columns=['대계열'])

In [ ]:
#재학생_성비, 정원내_입학자_학부_남
# 독립변수
X = univ_major.iloc[:, 43: ]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['재학생_성비']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                 재학생_성비   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     196.5
Date:                Wed, 23 Aug 2023   Prob (F-statistic):          7.00e-214
Time:                        07:26:51   Log-Likelihood:                -4067.1
No. Observations:                3221   AIC:                             8148.
Df Residuals:                    3214   BIC:                             8191.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1809      0.026     -7.001      0.0

**모든 독립변수로 모델 적합**

In [ ]:
all_independent = [col for col in numeric_columns_all if col != '표준점수']
print(all_independent)

['입학정원_학부_계', '정원내_입학자_학부_계', '정원내_입학자_학부_남', '정원내_입학자_학부_여', '전임교원_계', '모집인원', '최종_경쟁률', '재학생충원율', '취업률', '신입생충원율', '총교육비', '학생_1인당_교육비', '전임교원수', '재학생수', '전임교원확보율', '장학금총액', '학생_1인당_장학금', '모집군_가', '모집군_나', '모집군_다', '설립_국가기관', '설립_사립', '캠퍼스_본교', '캠퍼스_분교', '전체_재학생수_대', '전체_재학생수_소', '전체_재학생수_중', '계열_인문', '계열_자연', '학교명', '재학생_성비']


In [ ]:

# 독립변수

X = univ_df[all_independent]
X = sm.add_constant(X) # 상수항 추가

# 종속변수
y = univ_df['표준점수']

# 가정: X는 설명변수들, y는 종속변수, Z는 조절변수.


X = sm.add_constant(X)  # 절편을 추가합니다.
model = sm.OLS(y, X)  # 선형 회귀 모델을 생성합니다.
results = model.fit()  # 모델을 학습합니다.

print(results.summary())  # 모델 결과를 출력합니다.

                            OLS Regression Results                            
Dep. Variable:                   표준점수   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     312.5
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        01:29:22   Log-Likelihood:                -2533.0
No. Observations:                3221   AIC:                             5120.
Df Residuals:                    3194   BIC:                             5284.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.5018      0.018     27.390   

## 코드 작성란

아래에 코드를 작성해주세요.

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
# 고정효과 확인
#대계열 고정 모델
df_with_dummies_bmajor = pd.get_dummies(univ_df, columns=['대계열']) #다중공선성 방지

#학과 고정 모델
df_with_dummies_smajor = pd.get_dummies(univ_df, columns=['소계열']) #다중공선성 방지

#대학 고정 모델
df_with_dummies_univ = pd.get_dummies(univ_df, columns=['학교명']) #다중공선성 방지

In [ ]:
#대계열 고정 모델 적합
# 대계열에 따라 학교명(학교서열)와 표준점수 와의 관계가 다를 수 있음 ( ex) 의학등 특수계열) -> 이때 대계열을 더미로 변환 후 고정 시키고, 계열 내에서 학교명과 표준점수와의 관계 파악.

formula = '취업률 ~ 학교명 + '+'+'.join(df_with_dummies_bmajor.columns[-7:])
model = smf.ols(formula=formula, data=df_with_dummies_bmajor)
result = model.fit()

# 결과 출력
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    취업률   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     72.58
Date:                Wed, 23 Aug 2023   Prob (F-statistic):           8.15e-98
Time:                        08:36:47   Log-Likelihood:                -4334.0
No. Observations:                3221   AIC:                             8684.
Df Residuals:                    3213   BIC:                             8733.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9212      0.052     17.732      0.0

In [ ]:
df_with_dummies_smajor.columns = df_with_dummies_smajor.columns.str.replace('ㆍ', '_')
df_with_dummies_smajor.columns = df_with_dummies_smajor.columns.str.replace('·', '_')


In [ ]:
#대학 고정 모델 적합
formula = '취업률 ~ 학교명 +' + ' + '.join(df_with_dummies_smajor.columns[37:])
model = smf.ols(formula=formula, data=df_with_dummies_smajor)
result = model.fit()

# 결과 출력
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:                    취업률   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     7.307
Date:                Wed, 23 Aug 2023   Prob (F-statistic):           3.94e-88
Time:                        08:46:05   Log-Likelihood:                -4215.8
No. Observations:                3221   AIC:                             8644.
Df Residuals:                    3115   BIC:                             9288.
Df Model:                         105                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          1.0826      0.056     19.

In [ ]:
#학교 고정 모델 적합
formula = '취업률 ~ 총교육비 + '+'+'.join(df_with_dummies_univ.columns[37:])
model = smf.ols(formula=formula, data=df_with_dummies_univ)
result = model.fit()

# 결과 출력
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    취업률   R-squared:                       0.441
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     194.8
Date:                Wed, 23 Aug 2023   Prob (F-statistic):               0.00
Time:                        06:52:17   Log-Likelihood:                -3632.9
No. Observations:                3221   AIC:                             7294.
Df Residuals:                    3207   BIC:                             7379.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3287      0.021     15.682      0.0

## 결과 해석

아래에 해석한 결과를 작성해주세요.


**결과 해석** (추가해주세요)
1. 모집인원
-모집인원과 입결은 p-value가 0.000으로 유의한 양의 상관관계를 가진다.

2. 최종경쟁률
-최종경쟁률과 입결은 p-value가 0.011으로 유의한 음의 상관관계를 가진다.
-최종경쟁률과 학교명의 조절효과를 확인해보면 p-value가 0.000으로 음의 상관관계를 띄어 네임벨류가 높을수록 최종경쟁률의 증가하며 이는 최종경쟁률과 입결의 상관관계에도 영향을 미친다는 점을 알 수 있다.
-최종경쟁률과 취업률의 조절효과를 확인해보면 p-value가 0.000으로 음의 상관관계를 띄어 취업률이 높으면 최종경쟁률이 오히려 줄어들고 이는 최종경쟁률과 입결의 상관관계에도 영향을 미친다는 점을 알 수 있다. 취업률과 입결의 상관관계를 확인해 봤을때 p-value가 0.00으로 음의 상관관계를 가진다는 점에서 기인하여 취업률과 최종경쟁률의 음의 상관관계가 존재할 수 있다.

3. 재학생충원률
-재학생충원률과 입결은 p-value가 0.000으로 유의한 양의 상관관계를 가진다.

4. 총교육비
-총교육비와 입결은 p-value가 0.000으로 유의한 양의 상관관계를 가진다.

5. 학교명
-학교명과 입결은 p-value가 0.000으로 유의한 음의 상관관계를 가진다.
-이는 사회계열, 공학계열을 포함하는 대계열을 고정하여 고정효과를 알아봤을떄도 p-value가 0.000으로 계열에 상관없이 입결과 유의한 음의 상관관계를 띄는 것을 확인할 수 있다.

6. 재학생수
-재학생수와 입결은 p-value가 0.000으로 유의한 양의 상관관계를 가진다.

7. 전임교원확보율
-모집인원과 입결은 p-value가 0.000으로 유의한 양의 상관관계를 가진다.

8. 학생 1인당 장학금
-모집인원과 입결은 p-value가 0.000으로 유의한 음의 상관관계를 가진다.

9. 재학생 성비
-재학생 성비와 입결은 p-value가 0.000으로 유의한 음의 상관관계를 가진다.

____________
1. 전체 재학생 수 (대,중,소)
- 전체_재학생수_대와 표준점수는 p-value가 0.000으로 유의한 양의 상관관계를 가진다.
- 전체_재학생수_소와 표준점수는 p-value가 0.000으로 유의한 음의 상관관계를 가진다.
- 이는 대체로 재학생 수가 많은 학교 일수록 입결이 높은 학교임을 나타낸다.

2. 최종 경쟁률, 표준점수
- 최종 경쟁률와 표준점수는 p-value가 0.000으로 유의한 음의 상관관계를 가진다.
- 통상적으로 경쟁률이 높을수록 표준점수가 높을 것이라고 생각하지만, 분석 결과 그렇지 않았다. 최상위 대학은 입결과 경쟁률이 무의미함을 알 수 있었다.

3. 학교명, 전임교원수 , 총교육비
- 학교 네임벨류와 총교육비는 p-value가 0.000으로 유의한 음의 상관관계를 가진다.
- 전임교원수와 총교육비는 p-value가 0.009으로 유의한 음의 상관관계를 가진다.
- 전학교 네임벨류가 좋고 전임교원이 많은 학교는 실제로 교육비 투자를 많이 함을 알 수 있다.

4. 설립_국가기관, 장학금총액
- 설립_국가기관과 표준점수는 p-value가 0.000으로 유의한 양의 상관관계를 가진다. 대체로 국립 대학교는 표준점수가 높음을 보인다.
- 장학금총액과 표준점수는 는 p-value가 0.000으로 유의한 양의 상관관계를 가진다.
- 설립_국가기관:장학금총액과 표준점수는 는 p-value가 0.011으로 유의한 양의     
상관관계를 가진다. 이는 설립_국가기관이면서 장학금이 높을수록 표준점수가 높음을 알 수 있다.

5. 설립_사립, 장학금총액
- 설립_사립과 표준점수는 p-value가 0.000으로 유의한 음의 상관관계를 가진다. 대체로 사립대학교는 표준점수가 낮음을 보인다.
-  장학금총액과 표준점수는 p-value가 0.000으로 유의한 양의 상관관계를 가진다.
- 설립_사립:장학금총액과 표준점수는 는 p-value가 0.011으로 유의한 음의     
상관관계를 가진다. 이는 설립_사립이면서 장학금의 상관관계가 유의하게 약함을 알 수 있다.

##### 설립_국가기관, 사립을 비교하면서 국가기관을 선택한 사람은 장학금총액의 영향을 크게받는 반면, 사립을 선택한 사람들은 장학금총액의 영향을 크게 받지 않는다.

6. 대계열, 재학생 성비
- 공학, 자연계열과 재학생 성비는 p-value가 0.000으로 유의한 양의 상관관계를 가진다. 이는 공학, 자연계열에는 남학생이 많이 분포함을 알 수 있다.
- 인문, 의약, 교육계열과 재학생 성비는 p-value가 0.000으로 유의한 음의 상관관계를 가진다. 이는 인문, 의약, 교육계열에는 여학생이 많이 분포함을 알 수 있다.


----
# 고정 효과
취업률은 학교 네임벨류 뿐만 아니라, 대계열에 따라서도 영향을 받기때문에, 대계열을 고정시켜 학교 네임벨류에 따른 취업률을 분석했다.

1. 대계열 고정, 종속변수: 취업률, 독립변수:학교명
- 교육, 자연계열에서는 학교명과 취업률은 p-value가 0.000으로 유의한 음의 상관관계를 가진다. 이는 자연, 교육계열은 네임벨류가 좋은 학교일수록 취업률이 높음을 알 수 있다.
- 공학, 의약, 인문계열에서는 학교명과 취업률은 p-value가 0.000으로 유의한 양의 상관관계를 가진다. 이는 의약, 인문계열은 네임벨류와 취업률은 상관관계가 약함을 알 수 있다.



### 위 대계열을 세분화하여 분석
2. 소계열 고정, 종속변수: 취업률, 독립변수:학교명
- 소계열 중 간호, 복지 분야에서 학교명과 취업률은 p-value가 0.000으로 유의한 양의 상관관계를 가진다. 이는 간호, 복지 분야는 대학 네임벨류와 취업률은 상관관계가 약함을 알 수 있다.


3. 학교명 고정, 종속변수 취업률, 독립변수: 총교육비
- 상위 6레벨 학교까지는 p-value가 0.000으로 유의한 양의 상관관계를 가진다.
상위학교에서 총교육비 지출이 많을수록, 취업률이 높음을 알 수 있다.
예외적으로, 5레벨에 지방 거점 국립대가 포함되어 상관관계가 유의하지 않았다.
이는 지방 거정 국립대는 인서울 대학에 비해 인프라 부족으로 취업률이 낮음으로 유추할 수 있다.

- 상위 6레벨 미만 학교는 p-value가 0.050이하로 유의한 음의 상관관계를 가진다.
이는 하위학교에서 총교육비 지출과 취업률의 상관관계가 약함을 알 수 있다.